# <center> ALPS </center>

<font size="4"><br />This Jupyter Notebook provides the code for the following (Approximation by Localized Penalized Splines) ALPS framework functionalities:<br /><br />

1. [Using GCV to fit the ALPS model<br /><br />](#section_1)
1. [Using REML to fit the ALPS model<br /><br />](#section_2)
1. [Outlier detection in time series using two stage strategy<br /><br />](#section_3)
1. [Segregation of low frequency and high frequency effects<br /><br />](#section_4)
1. [Computation of first derivative for the produced approximation](#section_5)
</font>

In [1]:
# Importing the libraries
import os
import atexit
from importlib import reload
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
from IPython.display import HTML
import ipywidgets as widg

%matplotlib inline
plt.ioff()

import files
import functions as func
import plotting
import widgets

In [2]:
#prints functions.py to the functions_ouput window
functions_output = widg.Output()
functions_file = os.path.join(files.DIR_BIN, "functions.py")
def show_functions(change):

    if showmore_button.description == 'Show functions.py':
        
        showmore_button.description = 'Hide functions.py'
        with functions_output:
            f = open(functions_file, 'r')
            for line in f:
                print(line.rstrip())
            f.close()
    else:
        
        showmore_button.description = 'Show functions.py'
        functions_output.clear_output()
        
showmore_button = widg.Button(description = 'Show functions.py', button_style = 'success')
showmore_button.on_click(show_functions)


### Click the Hide / Show functions.py button to see the foundational functions on which these higher level ALPS framework functionalities are built

In [3]:
display(functions_output)
display(showmore_button)

Output()

Button(button_style='success', description='Show functions.py', style=ButtonStyle())

## View Your Data

Select *Sample* to choose a file from one of the included sample files. Select *Personal* to pick a file from your session's personal data directory. You can upload a CSV or pickle file using the *Upload* button. To load the selected data, press *Select* and a 5 row sample of your data will be shown.

In [4]:
d = widgets.DataSelector()
display(d)

DataSelector(children=(HBox(children=(Label(value='Select data source:'), Button(description='Sample', style=B…

<a name="section_1"></a>
## <br />1. Using GCV to fit the ALPS model[$\tiny\uparrow$](#top)

plt.close('all')

In [5]:
f1 = plt.figure(figsize=(12,7))

In [6]:
gcv = widgets.DataDisplay(widgets.DataDisplay.FuncType.GCV, f1)

DataSelector(children=(HBox(children=(Label(value='Select data source:'), Button(description='Sample', style=B…

<a name="section_2"></a>
## <br />2. Using REML to Fit the ALPS Model[$\tiny\uparrow$](#top)

In [7]:
reload(widgets)
reload(plotting)

<module 'plotting' from '/home/rimov/Documents/Projects/ghub/alps/ALPS2/bin/plotting.py'>

In [8]:
f2 = plt.figure(figsize=(9,5))

In [9]:
reml = widgets.DataDisplay(widgets.DataDisplay.FuncType.REML, f2)

DataDisplay(children=(DataSelector(children=(HBox(children=(Label(value='Select data source:'), Button(descrip…

<IPython.core.display.Javascript object>

<a name="section_3"></a>
## <br />3. Outlier Detection in Time Series using Two Stage Strategy[$\tiny\uparrow$](#top)

In [10]:
f3 = plt.figure(figsize=(15,5))

In [11]:
two_stage = widgets.DataDisplay(
    widgets.DataDisplay.FuncType.TWO_STAGE, f3
)

DataSelector(children=(HBox(children=(Label(value='Select data source:'), Button(description='Sample', style=B…

<a name="section_4"></a>
## <br />4. Mixed Model Formulation for getting Local and Global Effects[$\tiny\uparrow$](#top)

In [12]:
reload(files)

<module 'files' from '/home/rimov/Documents/Projects/ghub/alps/ALPS2/bin/files.py'>

In [13]:
f4 = plt.figure(figsize=(15,5))

In [14]:
mmf = widgets.DataDisplay(widgets.DataDisplay.FuncType.MMF, f4)

DataSelector(children=(HBox(children=(Label(value='Select data source:'), Button(description='Sample', style=B…

<a name="section_5"></a>
## <br />5. Computation of First Derivative with CIs[$\tiny\uparrow$](#top)

Here we show the capability of ALPS to compute the first order derivative. Actually its even possible to compute derivatives of higher order by changing the order in the main code.

In [15]:
data = files.load_pickle(files.DIR_SAMPLE_DATA/'ts1.p')

AttributeError: module 'files' has no attribute 'load_pickle'

In [ ]:
f5 = plt.figure(figsize=(9,5))

In [ ]:
# Fitting and plotting
f1 = figure(figsize=(15,10))
ax = subplot2grid((1,1),(0,0))

p = 4;q=2
[n,lamb,sigmasq] = func.full_search_nk(Data1,p,q)
c = n+p
U = func.Kno_pspline_opt(Data1,p,n)
B = func.Basis_Pspline(n,p,U,Data1[:,0])
P = func.Penalty_p(q,c)
theta = np.linalg.solve(B.T.dot(B) + lamb*P, B.T.dot(Data1[:,1].reshape(-1,1)))
### Getting mean of the prediction
num = 200
xpred = func.linspace(Data1[0,0],Data1[-1,0],num)
Bpred = func.Basis_Pspline(n,p,U,xpred)
ypred1 = Bpred.dot(theta)
std_t1,std_n1 = func.Var_bounds(Data1,Bpred,B,theta,P,lamb)
## Getting the derivative
Bpred_dert = func.Basis_derv_Pspline(n,p,U,xpred)
ypred_derth = Bpred_dert.dot(theta)
std_th_derv,std_nh_derv = func.Var_bounds(Data1,Bpred_dert,B,theta,P,lamb)


## Plotting
ax.scatter(Data1[:,0],Data1[:,1],color = 'r',s = 100,label = 'Data')
ax.plot(xpred,ypred1,linewidth=3,color = 'g',label = 'Mean Prediction')
ax.plot(xpred,ypred_derth,linewidth=3,color = 'b',label = 'First derivative')
ax.set_title('(a)',size = 25)
ax.tick_params(axis='x', labelsize=19)
ax.tick_params(axis='y', labelsize=19)
ax.set_xlabel('Time',size=25)
ax.set_ylabel('thickness change (m)',size = 25)
ax.fill_between(xpred.flatten(),ypred1.flatten()-std_t1,ypred1.flatten()+std_t1, alpha = 0.2,color = 'k',label = '95% t-interval')
ax.fill_between(xpred.flatten(),ypred_derth.flatten()-std_th_derv,ypred_derth.flatten()+std_th_derv, alpha = 0.2,color = 'k')


ax.legend(fontsize=20)
ax.grid(True)
show()


In [ ]:
# Prevent In[] and Out[] from displaying on left
HTML('''
<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>
''')

In [ ]:
# Scroll to top when the notebook is loaded
HTML('''
<script>
    function scroll_to_top() {
        Jupyter.notebook.scroll_to_top();
    } 
    $( window ).on( "load", scroll_to_top() );
</script>
''')